En esta notebook llevaremos a cabo el proceso de ETL para el archivo australian_users_items.json que se encuentra en la carpeta de data. Procederemos a leerlo, analizar nulos, registros vacios y de ser necesario eliminarlos o inputar valores, para terminar cargando los datos nuevamente a un archivo csv con la información limpia.

### Extracción de datos Users Items

In [9]:
import pandas as pd
import numpy as np
import json
import Tools as t
import warnings
warnings.filterwarnings("ignore")

Extraemos los datos del dataset, desanidamos columnas y los almacenamos en un dataframe

In [6]:
ruta_items = 'data/australian_users_items.json'
import ast

filas_items = []

with open(ruta_items, encoding='utf-8') as f:
    for line in f.readlines():
        try:
            data = json.loads(line)                                                                         # Intenta cargar el JSON normalmente
        except json.JSONDecodeError:                                                                        # Si tiene error:
            data = ast.literal_eval(line)                                                                   # Usa ast.literal_eval() para cargar el JSON inválido y cambiar '' por ""
        filas_items.append(data)                                                                            # Se agrega a una lista_items

df_items = pd.DataFrame(filas_items)                                                                        # Se pasa a un dataframe        
data_user_item1 = df_items.explode(['items'])                                                               # Desanidamos la columna items
data_user_item2 = pd.json_normalize(data_user_item1['items']).set_index(data_user_item1['items'].index)     # "Aplanamos" el dataframe
df_items2= pd.concat([data_user_item2, data_user_item1], axis=1)                                            # Unimos los df
df_items2

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url,items
0,10,Counter-Strike,6.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike..."
0,20,Team Fortress Classic,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ..."
0,30,Day of Defeat,7.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '30', 'item_name': 'Day of Defeat'..."
0,40,Deathmatch Classic,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '40', 'item_name': 'Deathmatch Cla..."
0,50,Half-Life: Opposing Force,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '50', 'item_name': 'Half-Life: Opp..."
...,...,...,...,...,...,...,...,...,...
88308,373330,All Is Dust,0.0,0.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '373330', 'item_name': 'All Is Dus..."
88308,388490,One Way To Die: Steam Edition,3.0,3.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '388490', 'item_name': 'One Way To..."
88308,521570,You Have 10 Seconds 2,4.0,4.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '521570', 'item_name': 'You Have 1..."
88308,519140,Minds Eyes,3.0,3.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '519140', 'item_name': 'Minds Eyes..."


Eliminamos la columna items, ya que ya fue desanidada y sus elementos anexados al dataframe

In [7]:
del df_items2['items']

#### Transformación de datos

Vemos que tipo de dato es cada variable, la cantidad de registros y columnas

In [8]:
df_items2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5170015 entries, 0 to 88309
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_id           object 
 1   item_name         object 
 2   playtime_forever  float64
 3   playtime_2weeks   float64
 4   user_id           object 
 5   items_count       int64  
 6   steam_id          object 
 7   user_url          object 
dtypes: float64(2), int64(1), object(5)
memory usage: 355.0+ MB


Calculamos la cantidad de valores nulos en el dataframe df_items2

In [11]:
nulos = t.PorcentajeNulos(df_items2)
nulos

,%_valores_nulos,Cantidad_Nulos,Cantidad_NO_Nulos,Total_Registros
item_id,0.33,16806,5153209,5170015
item_name,0.33,16806,5153209,5170015
playtime_forever,0.33,16806,5153209,5170015
playtime_2weeks,0.33,16806,5153209,5170015
user_id,0.00,0,5170015,5170015
items_count,0.00,0,5170015,5170015
steam_id,0.00,0,5170015,5170015
user_url,0.00,0,5170015,5170015


Eliminamos los registros nulos, ya que no son una cantidad significativa (menos del 2% del total)

In [8]:
df_items2 = df_items2.dropna()

Revisamos los duplicados

In [10]:
duplicados = df_items2[df_items2.duplicated(keep=False)]
duplicados.head()

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url
17,4000,Garry's Mod,269.0,0.0,76561198156664158,59,76561198156664158,http://steamcommunity.com/profiles/76561198156...
17,33910,Arma 2,162.0,0.0,76561198156664158,59,76561198156664158,http://steamcommunity.com/profiles/76561198156...
17,33930,Arma 2: Operation Arrowhead,223.0,0.0,76561198156664158,59,76561198156664158,http://steamcommunity.com/profiles/76561198156...
17,219540,Arma 2: Operation Arrowhead Beta (Obsolete),0.0,0.0,76561198156664158,59,76561198156664158,http://steamcommunity.com/profiles/76561198156...
17,400,Portal,196.0,0.0,76561198156664158,59,76561198156664158,http://steamcommunity.com/profiles/76561198156...


Al verificar si son resultados duplicados, observamos que las horas de playtime_forever y item_name son distintas a pesar que todo lo demás es igual, por lo que no se eliminarán porque no son realmente duplicados.

In [11]:
user_id = '76561198156664158'

fila_usuario = df_items2.loc[df_items2['user_id'] == user_id]
fila_usuario.head()

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url
17,4000,Garry's Mod,269.0,0.0,76561198156664158,59,76561198156664158,http://steamcommunity.com/profiles/76561198156...
17,33910,Arma 2,162.0,0.0,76561198156664158,59,76561198156664158,http://steamcommunity.com/profiles/76561198156...
17,33930,Arma 2: Operation Arrowhead,223.0,0.0,76561198156664158,59,76561198156664158,http://steamcommunity.com/profiles/76561198156...
17,219540,Arma 2: Operation Arrowhead Beta (Obsolete),0.0,0.0,76561198156664158,59,76561198156664158,http://steamcommunity.com/profiles/76561198156...
17,400,Portal,196.0,0.0,76561198156664158,59,76561198156664158,http://steamcommunity.com/profiles/76561198156...


Eliminaremos las columnas que no se utilizarán en los endpoints: playtime_2weeks, user_url

In [12]:
eliminar = ['playtime_2weeks', 'user_url']
df_items2 = df_items2.drop(columns=eliminar)

Ya que contamos con el dataframe limpio, proseguimos con la modificación del tipo de dato de cada columna

In [13]:
df_items2['user_id'] = df_items2['user_id'].astype(str)
df_items2['item_name'] = df_items2['item_name'].astype(str)
df_items2['user_id'] = df_items2['user_id'].astype(str)

### Carga de datos

In [ ]:
# Guardamos el conjunto ya limpio a un csv
df_items2.to_csv('users_items_cleaned.csv', index = False, encoding='utf-8')

Este es el fin de este ETL, porfavor da click [aqui](01c_ETL_reviews.ipynb) para continuar.